In [1]:
import h5py

# Open the HDF5 file
train_path = r'C:\Users\tulas\OneDrive\Desktop\RVU\Sem5\DL\Lab1\Train.h5'  # Replace with your file path
with h5py.File(train_path, 'r') as file:
    # Recursively print the structure of the file
    def print_structure(name, obj):
        if isinstance(obj, h5py.Group):
            print(f"Group: {name}")
        elif isinstance(obj, h5py.Dataset):
            print(f"Dataset: {name} - Shape: {obj.shape} - Data type: {obj.dtype}")

    file.visititems(print_structure)

Dataset: images - Shape: (2626, 128, 128, 3) - Data type: uint8
Dataset: labels - Shape: (2626,) - Data type: int32


In [2]:
import h5py

# Open the HDF5 file
test_path = r'C:\Users\tulas\OneDrive\Desktop\RVU\Sem5\DL\Lab1\Test.h5'  # Replace with your file path
with h5py.File(test_path, 'r') as file:
    # Recursively print the structure of the file
    def print_structure(name, obj):
        if isinstance(obj, h5py.Group):
            print(f"Group: {name}")
        elif isinstance(obj, h5py.Dataset):
            print(f"Dataset: {name} - Shape: {obj.shape} - Data type: {obj.dtype}")

    file.visititems(print_structure)

Dataset: images - Shape: (120, 128, 128, 3) - Data type: uint8
Dataset: labels - Shape: (120,) - Data type: int32


In [3]:
import h5py

def inspect_h5_file(file_path):
    with h5py.File(file_path, "r") as file:
        print("Keys in the file:", list(file.keys()))

# Inspect the training file
inspect_h5_file("Train.h5")

# Inspect the test file
inspect_h5_file("Test.h5")


Keys in the file: ['images', 'labels']
Keys in the file: ['images', 'labels']


In [4]:
import numpy as np
import h5py
import matplotlib.pyplot as plt

# Load the dataset
def load_data(train_path, test_path):
    train_dataset = h5py.File(train_path, "r")
    test_dataset = h5py.File(test_path, "r")
    
    train_x_orig = np.array(train_dataset["images"][:])  # Replace with the correct key
    train_y_orig = np.array(train_dataset["labels"][:])  # Replace with the correct key
    
    test_x_orig = np.array(test_dataset["images"][:])  # Replace with the correct key
    test_y_orig = np.array(test_dataset["labels"][:])  # Replace with the correct key
    
    return train_x_orig, train_y_orig, test_x_orig, test_y_orig

In [5]:
# Normalize the dataset
def preprocess_data(train_x_orig, test_x_orig):
    train_x_flatten = train_x_orig.reshape(train_x_orig.shape[0], -1).T
    test_x_flatten = test_x_orig.reshape(test_x_orig.shape[0], -1).T
    
    train_x = train_x_flatten / 255.
    test_x = test_x_flatten / 255.
    
    return train_x, test_x

In [6]:
# Softmax function
def softmax(z):
    exp_z = np.exp(z - np.max(z, axis=0, keepdims=True))
    return exp_z / np.sum(exp_z, axis=0, keepdims=True)

In [7]:
# Cost function for multi-class classification
def compute_cost(A, Y):
    m = Y.shape[1]
    cost = -np.sum(Y * np.log(A)) / m
    return cost

In [8]:
# Gradient Descent
def gradient_descent(X, Y, learning_rate, num_iterations):
    n_x, m = X.shape
    n_y = Y.shape[0]
    
    W = np.random.randn(n_y, n_x) * 0.01
    b = np.zeros((n_y, 1))
    
    for i in range(num_iterations):
        Z = np.dot(W, X) + b
        A = softmax(Z)
        
        cost = compute_cost(A, Y)
        
        dZ = A - Y
        dW = np.dot(dZ, X.T) / m
        db = np.sum(dZ, axis=1, keepdims=True) / m
        
        W -= learning_rate * dW
        b -= learning_rate * db
        
        if i % 100 == 0:
            print(f"Cost after iteration {i}: {cost}")
    
    return W, b

In [9]:
# Predict function
def predict(W, b, X):
    Z = np.dot(W, X) + b
    A = softmax(Z)
    predictions = np.argmax(A, axis=0)
    return predictions

In [10]:
# Convert labels to one-hot encoding
def convert_to_one_hot(Y, num_classes):
    Y_one_hot = np.eye(num_classes)[Y.reshape(-1)].T
    return Y_one_hot

In [11]:
# Load and preprocess the data
train_x_orig, train_y_orig, test_x_orig, test_y_orig = load_data("Train.h5", "Test.h5")
train_x, test_x = preprocess_data(train_x_orig, test_x_orig)

In [12]:
# Convert labels to one-hot encoding
num_classes = len(np.unique(train_y_orig))
train_y = convert_to_one_hot(train_y_orig, num_classes)
test_y = convert_to_one_hot(test_y_orig, num_classes)

In [13]:
# Train the model
learning_rate = 0.001
num_iterations = 5000
W, b = gradient_descent(train_x, train_y, learning_rate, num_iterations)

Cost after iteration 0: 2.083464296177889
Cost after iteration 100: 1.28923885425293
Cost after iteration 200: 1.2108923580991668
Cost after iteration 300: 1.1637875065670558
Cost after iteration 400: 1.1280831256858779
Cost after iteration 500: 1.098361420690051
Cost after iteration 600: 1.0724300417335393
Cost after iteration 700: 1.0491795217274689
Cost after iteration 800: 1.0279603837157922
Cost after iteration 900: 1.0083528965516984
Cost after iteration 1000: 0.99006650082975
Cost after iteration 1100: 0.9728899339039836
Cost after iteration 1200: 0.9566638961620838
Cost after iteration 1300: 0.941264840025503
Cost after iteration 1400: 0.9265947439975201
Cost after iteration 1500: 0.9125743465664475
Cost after iteration 1600: 0.8991385001383407
Cost after iteration 1700: 0.8862328858638808
Cost after iteration 1800: 0.873811634885109
Cost after iteration 1900: 0.8618355713588642
Cost after iteration 2000: 0.8502708923591535
Cost after iteration 2100: 0.8390881609624137
Cost aft

In [14]:
# Test the model
train_predictions = predict(W, b, train_x)
test_predictions = predict(W, b, test_x)

In [15]:
# Calculate accuracy
train_accuracy = np.mean(train_predictions == np.argmax(train_y, axis=0)) * 100
test_accuracy = np.mean(test_predictions == np.argmax(test_y, axis=0)) * 100

print(f"Train accuracy: {train_accuracy:.2f}%")
print(f"Test accuracy: {test_accuracy:.2f}%")

Train accuracy: 84.35%
Test accuracy: 43.33%
